In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import re

1. Start by scraping the data from the summary page for Tennessee's 2nd District, which is available at https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N.
    * The data that we want is contained in the "Total Raised and Spent" table.
    * Make a DataFrame showing, for each candidate:
        * the candidate's name
        * the candidate's party
        * state
        * district number
        * whether the candidate was an incumbent
        * whether the candidate won the race
        * the total amount raised by that candidate (as a numeric variable)
        * the total amount spent by the candidate (as a numeric variable)

In [10]:
URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N.'

tn2nd = requests.get(URL)

tn2ndsoup = BS(tn2nd.text)

tn2nd_table = tn2ndsoup.find('table', attrs={'class' : 'DataTable'})

tn2nd_df = (pd.read_html(str(tn2ndsoup.find('table', attrs={'class' : 'DataTable'}))))[0]



#regex for candidate, incumbent, party, and winner
candidate = r"(\w+\s\w+)"
incumbent = "Incumbent"
party = r"(\W\w\W)"
winner = "Winner"

#create dictionaries for columns with multiple info in them (candidate name has party, and  )
columns_extract= {'Candidate_name': candidate,'Party':party}
columns_contains = {'Incumbent':incumbent, 'Winner':winner}

#creates columns for party, incumbent, and winner and extracts the info from the candidate column
for key, value in columns_extract.items():
    tn2nd_df[key] = tn2nd_df['Candidate'].str.extract(value, expand=True)
    for key, value in columns_contains.items():
        tn2nd_df[key] = tn2nd_df['Candidate'].str.contains(value)

tn2nd_df = tn2nd_df[['Candidate_name', 
                                     'Party', 
                                     'Incumbent', 
                                     'Winner', 
                                     'Raised', 
                                     'Spent', 
                                     'Cash on Hand']]
#convert object types to int
tn2nd_df['Raised'] = tn2nd_df['Raised'].str.replace('\W', '')
tn2nd_df['Spent'] = tn2nd_df['Spent'].str.replace('\W', '')
tn2nd_df['Cash on Hand'] = tn2nd_df['Cash on Hand'].str.replace('\W', '')
tn2nd_df['Raised'] = tn2nd_df['Raised'].astype(int)
tn2nd_df['Spent'] = tn2nd_df['Spent'].astype(int)
tn2nd_df['Cash on Hand'] = tn2nd_df['Cash on Hand'].astype(int)

tn2nd_df

<ipython-input-10-24ce81f1f3d1>:37: FutureWarning: The default value of regex will change from True to False in a future version.
  tn2nd_df['Raised'] = tn2nd_df['Raised'].str.replace('\W', '')
<ipython-input-10-24ce81f1f3d1>:38: FutureWarning: The default value of regex will change from True to False in a future version.
  tn2nd_df['Spent'] = tn2nd_df['Spent'].str.replace('\W', '')
<ipython-input-10-24ce81f1f3d1>:39: FutureWarning: The default value of regex will change from True to False in a future version.
  tn2nd_df['Cash on Hand'] = tn2nd_df['Cash on Hand'].str.replace('\W', '')


,Candidate_name,Party,Incumbent,Winner,Raised,Spent,Cash on Hand
0,Tim Burchett,(R),True,True,1336276,878488,593678
1,Renee Hoyos,(D),False,False,812784,816793,210
